
![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/RE_ADE.ipynb)

# **Detect relations between Drugs and ADE**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload license_keys.json to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

# **Colab Setup**

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


# **🔎 For about models**


📌 **"re_ade_biobert"**--> *This model is capable of Relating Drugs and adverse reactions caused by them; It predicts if an adverse event is caused by a drug or not. It is based on ‘biobert_pubmed_base_cased’ embeddings. **1** : Shows the adverse event and drug entities are related, **0** : Shows the adverse event and drug entities are not related.*

📌 **"redl_ade_biobert"**--> *This model is an end-to-end trained BioBERT model, capable of Relating Drugs and adverse reactions caused by them; It predicts if an adverse event is caused by a drug or not. 1 : Shows the adverse event and drug entities are related, 0 : Shows the adverse event and drug entities are not related.*

📌 **"re_ade_clinical"**--> *This model is capable of Relating Drugs and adverse reactions caused by them; It predicts if an adverse event is caused by a drug or not. 1 : Shows the adverse event and drug entities are related, 0 : Shows the adverse event and drug entities are not related.*

🔎**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?task=Named+Entity+Recognition&edition=Spark+NLP+for+Healthcare)**






# **📌 re_ade_clinical**

### **🔎Define Spark NLP pipeline**

In [4]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

tokenizer = Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

ner_tagger = MedicalNerModel\
    .pretrained('ner_ade_clinical', "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_chunker = NerConverterInternal()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

reModel = RelationExtractionModel()\
    .pretrained('re_ade_clinical', "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(20)\
    .setRelationPairs(["drug-ade, ade-drug"])

pipeline = Pipeline(
    stages=[
        documenter,
        sentencer,
        tokenizer, 
        words_embedder, 
        pos_tagger, 
        ner_tagger,
        ner_chunker,
        dependency_parser,
        reModel
        ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)
light_model = LightPipeline(model)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
ner_ade_clinical download started this may take some time.
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
re_ade_clinical download started this may take some time.
Approximate size to download 10.9 MB
[OK!]


In [5]:
def get_relations_df (results, rel='relations'):
    rel_pairs=[]
    for rel in results[rel]:
        rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'],
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
        ))

    rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

    return rel_df[rel_df.relation!='O']

### **🔎Sample Text**

In [6]:
text = """A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""


### **🔎Run the pipeline**

In [7]:
import pandas as pd

light_result = light_model.fullAnnotate(text)
get_relations_df(light_result[0])

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,1,DRUG,25,32,naproxen,ADE,137,148,tense bullae,1.0
1,1,DRUG,25,32,naproxen,ADE,154,210,cutaneous fragility on the face and the back of the hands,0.99999964
2,1,DRUG,87,95,oxaprozin,ADE,137,148,tense bullae,1.0
3,1,DRUG,87,95,oxaprozin,ADE,154,210,cutaneous fragility on the face and the back of the hands,1.0


### **🔎Visualize results**

In [8]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = RelationExtractionVisualizer()

re_vis.display(light_result[0],
               relation_col = 'relations',
               document_col = 'document',
               show_relations=True
               )

# **📌redl_ade_biobert**

### **🔎Run the pipeline**

In [9]:
# Set a filter on pairs of named entities which will be treated as relation candidates
re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setMaxSyntacticDistance(20)\
    .setOutputCol("re_ner_chunks")\
    .setRelationPairs(['ade-drug', 'drug-ade'])

# The dataset this model is trained to is sentence-wise. 
# This model can also be trained on document-level relations - in which case, while predicting, use "document" instead of "sentence" as input.
re_model = RelationExtractionDLModel()\
    .pretrained('redl_ade_biobert', 'en', "clinical/models") \
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")


pipeline_dl = Pipeline(
    stages=[
        documenter,
        sentencer,
        tokenizer, 
        words_embedder, 
        pos_tagger, 
        ner_tagger,
        ner_chunker,
        dependency_parser,
        re_ner_chunk_filter,
        reModel
        ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model_dl = pipeline_dl.fit(empty_data)

redl_ade_biobert download started this may take some time.
[OK!]


### **🔎Sample Text**

In [10]:
text = """A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

### **🔎Run the pipeline**

In [11]:
light_result = light_model.fullAnnotate(text)
get_relations_df(light_result[0])

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,1,DRUG,25,32,naproxen,ADE,137,148,tense bullae,1.0
1,1,DRUG,25,32,naproxen,ADE,154,210,cutaneous fragility on the face and the back of the hands,0.99999964
2,1,DRUG,87,95,oxaprozin,ADE,137,148,tense bullae,1.0
3,1,DRUG,87,95,oxaprozin,ADE,154,210,cutaneous fragility on the face and the back of the hands,1.0


### **🔎Visualize results**

In [12]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = RelationExtractionVisualizer()

re_vis.display(light_result[0],
               relation_col = 'relations',
               document_col = 'document',
               show_relations=True
               )

# **📌re_ade_biobert**

In [13]:
words_embedder = BertEmbeddings \
    .pretrained("biobert_pubmed_base_cased") \
    .setInputCols(["sentences", "tokens"]) \
    .setOutputCol("embeddings")

ner_tagger = MedicalNerModel\
    .pretrained('ner_ade_biobert', "en", "clinical/models") \
    .setInputCols(["sentences", "tokens", "embeddings"]) \
    .setOutputCol("ner_tags")

reModel = RelationExtractionModel()\
    .pretrained('re_ade_biobert', "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(20)\
    .setPredictionThreshold(0.5)\
    .setRelationPairs(["drug-ade, ade-drug"])

pipeline_bert = Pipeline(
    stages=[
        documenter,
        sentencer,
        tokenizer, 
        words_embedder, 
        pos_tagger, 
        ner_tagger,
        ner_chunker,
        dependency_parser,
        reModel
        ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model_bert = pipeline_bert.fit(empty_data)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[OK!]
ner_ade_biobert download started this may take some time.
[OK!]
re_ade_biobert download started this may take some time.
Approximate size to download 17.1 MB
[OK!]


### **🔎Sample Text**

In [14]:
text = """A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

### **🔎Run the pipeline**

In [15]:
light_result = light_model.fullAnnotate(text)
get_relations_df(light_result[0])

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,1,DRUG,25,32,naproxen,ADE,137,148,tense bullae,1.0
1,1,DRUG,25,32,naproxen,ADE,154,210,cutaneous fragility on the face and the back of the hands,0.99999964
2,1,DRUG,87,95,oxaprozin,ADE,137,148,tense bullae,1.0
3,1,DRUG,87,95,oxaprozin,ADE,154,210,cutaneous fragility on the face and the back of the hands,1.0


### **🔎Visualize results**

In [16]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = RelationExtractionVisualizer()

re_vis.display(light_result[0],
               relation_col = 'relations',
               document_col = 'document',
               show_relations=True
               )